In [ ]:
#!/usr/bin/env python
# coding: utf-8

###################################### Librerias utilizadas ###################################

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input
#from keras.optimizer import adam_v2
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator     # Generador de imagenes.
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
import cv2
import os

get_ipython().run_line_magic('matplotlib', 'inline')

# Librerias para metricas de desempeño y matriz de confusion
from sklearn import metrics                                  # Funcion para el calculo de metricas de desempeño, la de abajo hace lo mismo pero todo por separado.
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, precision_score, recall_score, accuracy_score, roc_auc_score
from mlxtend.plotting import plot_confusion_matrix           # Paquete para ver matriz  de confusion en interfaz grafica.

import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

#Librerias para Generar data Augmentation
from keras.preprocessing.image import ImageDataGenerator 
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#Libreria para cargar modelo
from keras.models import load_model

# Libreria para obtener ruta de archivo actual
import pathlib

################################# Ingreso de parametros #########################################

pathInicial = str(pathlib.Path().absolute()) + "\\" # Path obtenido: ~/modelo

print("**** Ingreso de parametros ****\n")

tipoModelo = int(input("Tipo de modelo: \n Diurno --> Ingresar 0 \n Nocturno --> Ingresar 1\n"))
if not tipoModelo:
    tipoModelo = "Diurno"
else:
    tipoModelo = "Nocturno" 

modelVGG = int(input(" Modelo a utilizar: \n VGG16 --> Ingresar 0 \n VGG19 --> Ingresar 1\n"))
if not modelVGG:
    modelVGG = 16
else:
    modelVGG = 19 

epochs      = int(input(" Cantidad de epocas: \n"))
batch_size  = int(input(" Cantidad de muestras por actualización: \n"))
num_classes = int(input(" Número de clases: \n"))

################################# Configuracion de parametros ###################################

width_shape  = 224       # Ancho de imagen (Ancho tipico de VGG16 y VGG19)
height_shape = 224       # Alto  de imagen (Alto tipico de VGG16 y VGG19)
#num_classes  = 6        # Carpincho, Ciervo, Gato montés, Oveja, Puma, Vaca
#epochs       = 500      # Numero de epocas de entrenamiento
#batch_size   = 16       # Numero de muestras procesadas en cada actualizacion.
#modelVGG     = 16       # 16: VGG16 - 19: VGG19
#tipoModelo   = "Diurno" # "Diurno" o "Nocturno"

################################# Directorios ##################################################

model_dir               = pathInicial + r"model_VGG"+str(modelVGG)+"_"+str(epochs)+"E_"+str(batch_size)+"B_"+tipoModelo+".h5" 

if tipoModelo == "Diurno":
    train_data_dir      = pathInicial + r"imagenes_diurnas\Entrenamiento" # Path de entrenamiento
    validation_data_dir = pathInicial + r"imagenes_diurnas\Validacion"    # Path de validacion
    test_data_dir       = pathInicial + r"imagenes_diurnas\Test"          # Ruta de los datos a testear

else: #tipoModelo == "Nocturno"
    train_data_dir      = pathInicial + r"imagenes_nocturnas\Entrenamiento" # Path de entrenamiento
    validation_data_dir = pathInicial + r"imagenes_nocturnas\Validacion"    # Path de validacion
    test_data_dir       = pathInicial + r"imagenes_nocturnas\Test"          # Ruta de los datos a testear

############################### Excepciones ####################################################
    
cantTrainData = len(os.listdir(train_data_dir+"\\"+"Carpincho"+"\\"))      * len(os.listdir(train_data_dir+"\\"+"Ciervo"+"\\"))      * len(os.listdir(train_data_dir+"\\"+"Gato montes"+"\\"))      * len(os.listdir(train_data_dir+"\\"+"Oveja"+"\\"))      * len(os.listdir(train_data_dir+"\\"+"Puma"+"\\"))      * len(os.listdir(train_data_dir+"\\"+"Vaca"+"\\"))
cantValData   = len(os.listdir(validation_data_dir+"\\"+"Carpincho"+"\\")) * len(os.listdir(validation_data_dir+"\\"+"Ciervo"+"\\")) * len(os.listdir(validation_data_dir+"\\"+"Gato montes"+"\\")) * len(os.listdir(validation_data_dir+"\\"+"Oveja"+"\\")) * len(os.listdir(validation_data_dir+"\\"+"Puma"+"\\")) * len(os.listdir(validation_data_dir+"\\"+"Vaca"+"\\"))
cantTestData  = len(os.listdir(test_data_dir+"\\"+"Carpincho"+"\\"))       * len(os.listdir(test_data_dir+"\\"+"Ciervo"+"\\"))       * len(os.listdir(test_data_dir+"\\"+"Gato montes"+"\\"))       * len(os.listdir(test_data_dir+"\\"+"Oveja"+"\\"))       * len(os.listdir(test_data_dir+"\\"+"Puma"+"\\"))       * len(os.listdir(test_data_dir+"\\"+"Vaca"+"\\"))

if (cantTrainData == 0) or (cantValData == 0) or (cantTestData == 0):
    raise ValueError("Uno o varios de los directorios de datos del modelo " +tipoModelo+" se encuentran vacios (Entrenamiento, Test o Validación)") 
    
################################## Data augmentation ###############################################

# Generador de imagenes de entrenamiento para tener mayor cantidad de imagenes
train_datagen = ImageDataGenerator(  
    rotation_range         = 20,                # Rotacion de 20 grados
    zoom_range             = 0.2,               # zoom de 0.2 (20%)
    width_shift_range      = 0.1,               # Desplazamiento horizontal
    height_shift_range     = 0.1,               # Vertical 
    horizontal_flip        = True,              # Rotacion horizontal
    vertical_flip          = False,             # Rotacion vertical
    preprocessing_function = preprocess_input)

# Generador de imagenes de validacion para tener mayor cantidad de imagenes
valid_datagen = ImageDataGenerator(    
    rotation_range         = 20,
    zoom_range             = 0.2,
    width_shift_range      = 0.1,
    height_shift_range     = 0.1,
    horizontal_flip        = True,
    vertical_flip          = False,
    preprocessing_function = preprocess_input) 

# Direccion de imagenes de entrenamiento
train_generator  = train_datagen.flow_from_directory(  
    train_data_dir,                             # Direccion de imagenes de entrenamiento.
    target_size  = (width_shape, height_shape), # Dimensiones de imagenes de entrenamiento.
    batch_size   = batch_size,
    #save_to_dir ='',                           # Aca ponemos una direccion si queremos guardar las imagenes transformadas y generadas.
    class_mode   ='categorical')                # Tipo de clase, al ser mas de 2 pasa de ser binario a categorical.

# Direccion de imagenes de entrenamiento
validation_generator = valid_datagen.flow_from_directory(  
    validation_data_dir,                              
    target_size          = (width_shape, height_shape),          
    batch_size           = batch_size,
    #save_to_dir         = '',             Aca ponemos una direccion si queremos guardar las imagenes transformadas y generadas.
    class_mode           = 'categorical')

##################################### Entrenamiento ################################################

# Modelo preentrenado a ejecutar

nb_train_samples      = train_generator.samples            # Numero de imagenes de entrenamiento.
nb_validation_samples = validation_generator.samples       # Numero de imagenes de validacion.  

image_input = Input(shape = (width_shape, height_shape, 3)) #Importamos imagenes con formato: Ancho x alto x 3 canales RGB

if  modelVGG == 16:
    model = VGG16(input_tensor = image_input, include_top=True,weights='imagenet') #Cargamos modelo con parametros:
                                                                                                    # Imagen de entrada
                                                                                                    # Incluye ultima capa
                                                                                                    # Conserva pesos preentrenado de dataset imagenet
else: #VGG19
    model = VGG19(input_tensor = image_input, include_top=True,weights='imagenet')
    
last_layer = model.get_layer('fc2').output                                     # Tomamos ultima capa del modelo
out = Dense(num_classes, activation ='softmax', name ='output')(last_layer)    # Agregamos a ultima capa una capa densa que tiene como salida el numero de clases.
custom_vgg_model = Model(image_input, out)                                     # compilamos el modelo.

for layer in custom_vgg_model.layers[:-1]:                                     # Transfer learning utilizado para tomar 'pesos' de otros modelos ya entrenados con 
	layer.trainable = False                                                    # distitos dataset de imagenes. Solo entrenaremos la ultima capa.
                                                   

custom_vgg_model.compile(loss = 'categorical_crossentropy',optimizer = 'adadelta',metrics = ['accuracy'])   # Compilamos modelo, con funcion loss, optimizador y metrica para el entrenamiento.

custom_vgg_model.summary()                                                     # Utilizado para ver la estructura del modelo

# Solo entrenamos la ultima capa del summary, la cual posee 4097 parametros por cantidad de clase, conservando el preentrenamiento
# que trae el modelo de manera predefinida.

model_history = custom_vgg_model.fit_generator(   
    train_generator,                                 
    epochs           = epochs,
    validation_data  = validation_generator,
    steps_per_epoch  = nb_train_samples//batch_size,
    validation_steps = nb_validation_samples//batch_size)                      # Etapa de entrenamiento

custom_vgg_model.save(model_dir)           # Guardado de modelo

########################## Graficas de entrenamiento y validacion ####################################

def plotTraining(hist, epochs, typeData):
    
    if typeData == "loss":
        plt.figure(1,figsize=(10,5))
        yc = hist.history['loss']
        xc = range(epochs)
        plt.ylabel('Perdida', fontsize=24)
        plt.plot(xc,yc,'-r',label='Perdida en entrenamiento')
        
    if typeData=="accuracy":
        plt.figure(2,figsize=(10,5))
        yc = hist.history['accuracy']
        for i in range(0, len(yc)):
            yc[i]=100*yc[i]
        xc = range(epochs)
        plt.ylabel('Precisión (%)', fontsize=24)
        plt.plot(xc,yc,'-r',label='Precisión en entrenamiento')
        
    if typeData == "val_loss":
        plt.figure(1,figsize=(10,5))
        yc = hist.history['val_loss']
        xc = range(epochs)
        plt.ylabel('Perdida', fontsize=24)
        plt.plot(xc,yc,'--b',label='Perdida en validación')
        
    if typeData == "val_accuracy":
        plt.figure(2,figsize=(10,5))
        yc = hist.history['val_accuracy']
        for i in range(0, len(yc)):
            yc[i]=100*yc[i]
        xc = range(epochs)
        plt.ylabel('Precisión (%)', fontsize=24)
        plt.plot(xc,yc,'--b',label='Precisión en validación')
        

    plt.rc('xtick',labelsize=24)
    plt.rc('ytick',labelsize=24)
    plt.rc('legend', fontsize=18) 
    plt.legend()
    plt.xlabel('Numero de epocas',fontsize=24)
    plt.grid(True)

# Plot history:  Graficamos valores obtenidos en el entrenamiento

plotTraining(model_history,epochs,"loss")                
plotTraining(model_history,epochs,"accuracy")
plotTraining(model_history,epochs,"val_loss")
plotTraining(model_history,epochs,"val_accuracy")

############################## Metricas de desempeño y matriz de confusion ##########################

custom_Model = load_model(model_dir)

names = ['Carpincho','Ciervo','Gato montes','Oveja','Puma','Vaca'] # Etiquetas para la matriz

test_datagen  = ImageDataGenerator()                                # Generacion de imagenes

test_generator = test_datagen.flow_from_directory(                 
    test_data_dir,
    target_size   = (width_shape, height_shape), 
    batch_size    = batch_size,
    class_mode    = 'categorical', 
    shuffle       = False)           #Para que cargue el dataset en orden

preds  = custom_Model.predict_generator(generator=test_generator)   # Realizacion de la prediccion

y_pred = np.argmax(preds, axis=1)                                   # Se asigna clase segun valor maximo de probabilidad.
y_real = test_generator.classes                                     # Clase REAL.

matc   = confusion_matrix(y_real, y_pred)                           # Generacion de matriz de prediccion.

plot_confusion_matrix(conf_mat=matc, figsize=(9,9), class_names = names, show_normed=False)
plt.tight_layout()

print(metrics.classification_report(y_real,y_pred, digits = 4))
